# Analyzing FEMA's National Flood Insurance Program (NFIP) Data With DuckDB
Author: Mark Bauer

# OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2. Retrieved from https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about OpenFEMA's  [Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

**Dataset Description**:
>This dataset provides details on NFIP claims transactions. It is derived from the NFIP system of record, staged in the NFIP reporting platform and redacted to protect policy holder personally identifiable information.
>
>This dataset is not intended to be an official federal report, and should not be considered an official federal report.

**About the National Flood Insurance Program**:   
>Congress passed the National Flood Insurance Act (NFIA), 42 U.S.C. 4001 in 1968, creating the National Flood Insurance Program (NFIP) in order to reduce future flood losses through flood hazard identification, floodplain management, and providing insurance protection. The Department of Housing and Urban Development (HUD) originally administered the NFIP, and Congress subsequently transferred the NFIP to FEMA upon its creation in 1979. FEMA and insurance companies participating in FEMA's Write Your Own (WYO) program offer NFIP insurance coverage for building structures as well as for contents and personal property within the building structures to eligible and insurable properties. The WYO program began in 1983 with NFIP operating under Part B of the NFIA and allows FEMA to authorize private insurance companies to issue the Standard Flood Insurance Policy (SFIP) as FEMA's fiduciary and fiscal agent. FEMA administers NFIP by ensuring insurance applications are processed properly; determining correct premiums; renewing, reforming, and cancelling insurance policies; transferring policies from the seller of the property to the purchaser of the property in certain circumstances; and processing insurance claims.
>
>The paid premiums of SFIPs and claims payments for damaged property are processed through the National Flood Insurance Fund (NFIF). NFIF was established by the National Flood Insurance Act of 1968 (42 U.S.C. 4001, et seq.), and is a centralized premium revenue and fee-generated fund that supports NFIP, which holds these U.S. Treasury funds.
>
>The Flood Insurance Claims Manual (https://nfipservices.floodsmart.gov/insurance-manuals) provides claims guidance to WYOs, vendors, adjusters, and examiners so that policyholders experience consistent and reliable service. The Manual provides processes for handling claims from the notice of loss to final payment. The NFIP has provided answers to Frequently Asked Questions (FAQs) to assist the public in understanding and navigating the data our program makes available: https://www.fema.gov/sites/default/files/documents/fema_nfip-data-faqs.pdf.

**Data Dictionary**:  
View the data dictionary on OpenFEMA under the [Data Fields](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2) section.

Note: In this notebook, all dollar amounts are reported in nominal terms and have not been adjusted for inflation.

In [1]:
# import libraries
import duckdb
import pandas as pd

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb,pandas

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0
pandas: 1.5.1



In [3]:
# list datasets
%ls data/

FimaNfipClaims.parquet


# Redacted Claims: Before Getting Started
Please note that this dataset provides details on NFIP claims transactions and is ***redacted*** to protect policy holder personally identifiable information. The claim's `latitude` and `longitude` field **should not be used to represent the precise location of the insured building**. From the data dictionary:

>Latitude: **Approximate latitude of the insured building (to 1 decimal place)**. This represents the approximate location of the insured property. The precision has been lessened to ensure individual privacy. This may result in a point location that exists in an incorrect county or state. Use the state and county fields for record aggregation for these dimensions.
>
>Longitude: **Approximate longitude of the insured building (to 1 decimal place)**. This represents the approximate location of the insured property. The precision has been lessened to ensure individual privacy. This may result in a point location that exists in an incorrect county or state. Use the state and county fields for record aggregation for these dimensions.

For more information, visit the [Data Dictionary](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2) and review [Frequently Asked Questions about NFIP Policies and Claims Data](https://nfipservices.floodsmart.gov/frequently-asked-questions-about-nfip-policies-and-claims-data).

This analysis presents all financial figures in nominal dollars. No adjustments for inflation have been made, meaning that the values reflect the actual dollar amounts at the time of measurement. Additionally, the date and time of data access can be found in the download-data notebook.

# Create a DuckDB database instance using the Python client

In [4]:
# create a DuckDB database instance
con = duckdb.connect()

# create table claims of dataset
con.execute("""
    CREATE TABLE claims AS
        FROM read_parquet('data/FimaNfipClaims.parquet')
""")

# sanity check
con.sql("""
    SELECT *
    FROM claims
    LIMIT 5
""").show()

┌──────────────────────┬──────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ … │   latitude   │  longitude   │          id          │
│       boolean        │ timestamp with tim…  │   │ decimal(9,1) │ decimal(9,1) │         uuid         │
├──────────────────────┼──────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ false                │ 2020-12-11 11:25:4…  │ … │         30.3 │        -87.7 │ ac2b2b72-4b71-4415…  │
│ false                │ 2020-11-13 09:50:3…  │ … │         30.3 │        -87.7 │ dd95560b-afd1-46cb…  │
│ false                │ 2022-04-21 15:56:4…  │ … │         32.5 │        -86.4 │ e434f7e3-eeff-42e4…  │
│ false                │ 2020-03-09 15:28:2…  │ … │         32.5 │        -86.4 │ 5ddd1b02-a847-4b20…  │
│ false                │ 2020-01-22 11:55:5…  │ … │         32.5 │        -86.5 │ a6a39c3c-8ee5-47e9…  │
├──────────────────────┴──────────────────────┴───┴────

In [5]:
# list tables and schemas
con.sql("SHOW ALL TABLES").df()

,database,schema,name,column_names,column_types,temporary
0,memory,main,claims,"[agricultureStructureIndicator, asOfDate, base...","[BOOLEAN, TIMESTAMP WITH TIME ZONE, SMALLINT, ...",False


In [6]:
# count of rows
con.sql("""
    SELECT COUNT(*) AS count_rows
    FROM claims
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│    2709121 │
└────────────┘

In [7]:
# count of columns
con.sql("""
    SELECT COUNT(column_name) AS count_columns
    FROM (DESCRIBE FROM claims)
""")

┌───────────────┐
│ count_columns │
│     int64     │
├───────────────┤
│            73 │
└───────────────┘

In [8]:
# last refreshed
con.sql("""
    SELECT asOfDate
    FROM claims
    ORDER BY asOfDate DESC
    LIMIT 1
""")

┌───────────────────────────────┐
│           asOfDate            │
│   timestamp with time zone    │
├───────────────────────────────┤
│ 2025-04-04 18:15:24.042001-04 │
└───────────────────────────────┘

# Examine Dataset

## Column Info

In [9]:
# examine column datatypes
con.sql("""
    SELECT column_name, column_type
    FROM (DESCRIBE claims)
""").show(max_rows=80)

┌────────────────────────────────────────────┬──────────────────────────┐
│                column_name                 │       column_type        │
│                  varchar                   │         varchar          │
├────────────────────────────────────────────┼──────────────────────────┤
│ agricultureStructureIndicator              │ BOOLEAN                  │
│ asOfDate                                   │ TIMESTAMP WITH TIME ZONE │
│ basementEnclosureCrawlspaceType            │ SMALLINT                 │
│ policyCount                                │ SMALLINT                 │
│ crsClassificationCode                      │ SMALLINT                 │
│ dateOfLoss                                 │ DATE                     │
│ elevatedBuildingIndicator                  │ BOOLEAN                  │
│ elevationCertificateIndicator              │ VARCHAR                  │
│ elevationDifference                        │ DECIMAL(6,1)             │
│ baseFloodElevation                  

In [10]:
# approximate column null percentage
con.sql("""
    SELECT
        column_name,
        null_percentage
    FROM
        (SUMMARIZE FROM claims)
    ORDER BY
        null_percentage DESC
""").show(max_rows=80)

┌────────────────────────────────────────────┬─────────────────┐
│                column_name                 │ null_percentage │
│                  varchar                   │  decimal(9,2)   │
├────────────────────────────────────────────┼─────────────────┤
│ floodCharacteristicsIndicator              │           98.53 │
│ eventDesignationNumber                     │           93.23 │
│ lowestAdjacentGrade                        │           81.10 │
│ crsClassificationCode                      │           79.69 │
│ nonPaymentReasonBuilding                   │           77.93 │
│ elevationCertificateIndicator              │           77.92 │
│ lowestFloorElevation                       │           76.36 │
│ baseFloodElevation                         │           75.67 │
│ elevationDifference                        │           72.95 │
│ floodZoneCurrent                           │           71.72 │
│ nfipCommunityNumberCurrent                 │           71.68 │
│ basementEnclosureCrawls

## Preview Data

In [11]:
# preview data
con.sql("""
    SELECT *
    FROM claims
    LIMIT 5
""").show()

┌──────────────────────┬──────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ … │   latitude   │  longitude   │          id          │
│       boolean        │ timestamp with tim…  │   │ decimal(9,1) │ decimal(9,1) │         uuid         │
├──────────────────────┼──────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ false                │ 2020-12-11 11:25:4…  │ … │         30.3 │        -87.7 │ ac2b2b72-4b71-4415…  │
│ false                │ 2020-11-13 09:50:3…  │ … │         30.3 │        -87.7 │ dd95560b-afd1-46cb…  │
│ false                │ 2022-04-21 15:56:4…  │ … │         32.5 │        -86.4 │ e434f7e3-eeff-42e4…  │
│ false                │ 2020-03-09 15:28:2…  │ … │         32.5 │        -86.4 │ 5ddd1b02-a847-4b20…  │
│ false                │ 2020-01-22 11:55:5…  │ … │         32.5 │        -86.5 │ a6a39c3c-8ee5-47e9…  │
├──────────────────────┴──────────────────────┴───┴────

In [12]:
# preview data as pandas dataframe for readability
sql = """
    SELECT *
    FROM claims
    LIMIT 5
"""

# examine each column in sections because of large number of columns
con.sql(sql).df().iloc[:, :15]

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation
0,False,2020-12-11 11:25:40.587000-05:00,0.0,1,NaN,2020-09-16,True,None,4.0,10.0,AE,False,7.0,4.4,14.4
1,False,2020-11-13 09:50:38.288000-05:00,2.0,1,NaN,2020-09-16,False,None,6.0,10.0,AE,False,NaN,0.0,16.0
2,False,2022-04-21 15:56:43.051000-04:00,NaN,1,8.0,2020-03-05,True,None,NaN,NaN,X,False,7.0,NaN,NaN
3,False,2020-03-09 15:28:27.372000-04:00,NaN,1,8.0,2017-06-22,True,None,NaN,NaN,X,False,7.0,NaN,NaN
4,False,2020-01-22 11:55:53.194965-05:00,NaN,1,8.0,2016-01-22,False,None,NaN,NaN,X,False,NaN,NaN,NaN


In [13]:
# slice through columns
con.sql(sql).df().iloc[:, 15:30]

,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss
0,3,False,54.0,1,1991-09-17,2017-10-05,2750.18,121.0,0.0,True,8,False,10800,700,2020
1,2,False,NaN,2,1975-01-01,2008-10-20,2695.44,0.0,0.0,True,8,False,7400,0,2020
2,2,False,NaN,1,1968-01-03,2017-02-02,NaN,NaN,NaN,False,7,False,30000,12000,2020
3,2,False,NaN,1,1968-01-03,2017-02-02,13203.99,0.0,0.0,False,7,False,30000,12000,2017
4,1,False,NaN,1,1970-01-01,2015-10-20,NaN,NaN,NaN,False,R,False,250000,100000,2016


In [14]:
# slice through columns
con.sql(sql).df().iloc[:, 30:45]

,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator
0,True,4000.0,F,2750.18,278027.0,1,N,1371.0,F,121.0,50000.0,0,AL0520,NaN,NaN
1,False,3945.0,F,2695.44,167531.0,1,N,0.0,None,0.0,0.0,0,AL0520,NaN,NaN
2,True,948.0,1,0.00,92418.0,4,N,NaN,1,0.0,NaN,0,None,NaN,NaN
3,True,14204.0,1,13203.99,164992.0,2,N,NaN,1,0.0,NaN,0,None,NaN,NaN
4,True,NaN,F,0.00,NaN,0,N,NaN,F,0.0,NaN,0,None,NaN,NaN


In [15]:
# slice through columns
con.sql(sql).df().iloc[:, 45:60]

,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator
0,NaN,False,Hurricane Sally,30000,0.0,015005,0000,None,None,None,0,314835.0,75000.0,A,False
1,NaN,False,Hurricane Sally,30000,0.0,015005,0000,None,None,None,0,185457.0,0.0,A,False
2,NaN,False,None,30000,0.0,010002,010002,"PRATTVILLE, CITY OF",06,01,1,293079.0,NaN,A,False
3,0.0,False,None,30000,0.0,010002,010002,"PRATTVILLE, CITY OF",97,None,1,214275.0,NaN,A,False
4,0.0,False,None,30000,0.0,010002,010002,"PRATTVILLE, CITY OF",06,06,1,NaN,NaN,A,False


In [16]:
# slice through columns
con.sql(sql).df().iloc[:, 60:]

,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0.0,AE,1,False,AL,Currently Unavailable,36542,01003,01003011407,010030114072,30.3,-87.7,ac2b2b72-4b71-4415-966f-162f6f4feb6c
1,0.0,None,1,False,AL,Currently Unavailable,36542,01003,01003011407,010030114073,30.3,-87.7,dd95560b-afd1-46cb-b76a-19c58f2d9627
2,99.0,X,1,False,AL,Currently Unavailable,36066,01001,01001020400,010010204001,32.5,-86.4,e434f7e3-eeff-42e4-abc6-a5d49a1c84be
3,NaN,X,1,False,AL,Currently Unavailable,36066,01001,01001020400,010010204001,32.5,-86.4,5ddd1b02-a847-4b20-87be-5802da962064
4,0.0,AE,1,False,AL,Currently Unavailable,36067,01001,01001020600,010010206001,32.5,-86.5,a6a39c3c-8ee5-47e9-975f-6ad066a3adb7


In [17]:
# count duplicate IDs
con.sql("""
    SELECT
        id,
        COUNT(id) AS count
    FROM
        claims
    GROUP BY
        id
    HAVING
        count > 1
""").show()

┌──────┬───────┐
│  id  │ count │
│ uuid │ int64 │
├──────┴───────┤
│    0 rows    │
└──────────────┘



In [18]:
# earliest and latest record effective date
con.sql("""
    SELECT
        min(asOfDate) AS earliestAsOfDate,
        max(asOfDate) AS latestAsOfDate
    FROM
        claims
""").show()

┌────────────────────────────┬───────────────────────────────┐
│      earliestAsOfDate      │        latestAsOfDate         │
│  timestamp with time zone  │   timestamp with time zone    │
├────────────────────────────┼───────────────────────────────┤
│ 2019-09-19 02:12:43.388-04 │ 2025-04-04 18:15:24.042001-04 │
└────────────────────────────┴───────────────────────────────┘



In [19]:
# top 5 most recent effective date claim records
con.sql("""
    SELECT
        asOfDate,
        dateOfLoss,
        floodEvent,
        state,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim)::BIGINT AS paidTotalClaim
    FROM
        claims
    ORDER BY
        asOfDate DESC
    LIMIT 5
""").show()

┌───────────────────────────────┬────────────┬────────────────────────────┬─────────┬────────────────┐
│           asOfDate            │ dateOfLoss │         floodEvent         │  state  │ paidTotalClaim │
│   timestamp with time zone    │    date    │          varchar           │ varchar │     int64      │
├───────────────────────────────┼────────────┼────────────────────────────┼─────────┼────────────────┤
│ 2025-04-04 18:15:24.042001-04 │ 2025-03-28 │ NULL                       │ TX      │          25000 │
│ 2025-04-04 18:12:18.686999-04 │ 2024-10-10 │ Hurricane Milton           │ FL      │          23998 │
│ 2025-04-04 18:12:16.152699-04 │ 2025-02-15 │ NULL                       │ VA      │          63003 │
│ 2025-04-04 18:11:18.694396-04 │ 2025-02-15 │ February Kentucky Flooding │ KY      │          65999 │
│ 2025-04-04 18:08:12.322773-04 │ 2025-03-27 │ NULL                       │ TX      │          25000 │
└───────────────────────────────┴────────────┴───────────────────────────

In [20]:
# earliest and latest date of loss in dataset
con.sql("""
    SELECT
        min(dateOfLoss) AS earliestDateOfLoss,
        max(dateOfLoss) AS latestDateOfLoss
    FROM
        claims
""").show()

┌────────────────────┬──────────────────┐
│ earliestDateOfLoss │ latestDateOfLoss │
│        date        │       date       │
├────────────────────┼──────────────────┤
│ 1978-01-01         │ 2025-04-04       │
└────────────────────┴──────────────────┘



In [21]:
# top 5 latest claim records by date of loss 
con.sql("""
    SELECT
        dateOfLoss,
        asOfDate,
        floodEvent,
        state,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim)::BIGINT AS paidTotalClaim
    FROM
        claims
    ORDER BY
        dateOfLoss DESC,
        asOfDate DESC
    LIMIT 5
""").show()

┌────────────┬───────────────────────────────┬────────────┬─────────┬────────────────┐
│ dateOfLoss │           asOfDate            │ floodEvent │  state  │ paidTotalClaim │
│    date    │   timestamp with time zone    │  varchar   │ varchar │     int64      │
├────────────┼───────────────────────────────┼────────────┼─────────┼────────────────┤
│ 2025-04-04 │ 2025-04-04 17:34:27.985671-04 │ NULL       │ KY      │           NULL │
│ 2025-04-04 │ 2025-04-04 17:34:27.043969-04 │ NULL       │ KY      │           NULL │
│ 2025-04-04 │ 2025-04-04 16:34:45.412576-04 │ NULL       │ KY      │           NULL │
│ 2025-04-04 │ 2025-04-04 16:24:18.656965-04 │ NULL       │ KY      │           NULL │
│ 2025-04-04 │ 2025-04-04 16:14:49.92856-04  │ NULL       │ KY      │           NULL │
└────────────┴───────────────────────────────┴────────────┴─────────┴────────────────┘



In [22]:
# top 5 latest claim records by date of loss where total paid claim > 0
con.sql("""
    SELECT
        dateOfLoss,
        asOfDate,
        floodEvent,
        state,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim)::BIGINT AS paidTotalClaim
    FROM
        claims
    WHERE
        paidTotalClaim > 0
    ORDER BY
        dateOfLoss DESC,
        asOfDate DESC
    LIMIT 5
""").show()

┌────────────┬───────────────────────────────┬────────────┬─────────┬────────────────┐
│ dateOfLoss │           asOfDate            │ floodEvent │  state  │ paidTotalClaim │
│    date    │   timestamp with time zone    │  varchar   │ varchar │     int64      │
├────────────┼───────────────────────────────┼────────────┼─────────┼────────────────┤
│ 2025-03-30 │ 2025-04-04 11:44:10.640913-04 │ NULL       │ LA      │          20000 │
│ 2025-03-30 │ 2025-04-04 11:12:57.873346-04 │ NULL       │ LA      │          10000 │
│ 2025-03-29 │ 2025-04-04 10:29:13.542585-04 │ NULL       │ LA      │          40000 │
│ 2025-03-29 │ 2025-04-03 17:22:28.501115-04 │ NULL       │ LA      │          15000 │
│ 2025-03-29 │ 2025-04-03 06:17:04.131733-04 │ NULL       │ LA      │          11500 │
└────────────┴───────────────────────────────┴────────────┴─────────┴────────────────┘



In [23]:
# total insured units in dataset
con.sql("""
    SELECT SUM(policyCount) AS totalPolicyCount
    FROM claims
""").show()

┌──────────────────┐
│ totalPolicyCount │
│      int128      │
├──────────────────┤
│          3543787 │
└──────────────────┘



Policy Count:
>Insured units in an active status. A policy contract ceases to be in an active status as of the cancellation date or the expiration date. Residential Condominium Building Association Policy (RCBAP) contracts are stored as a single policy contract but insure multiple units and therefore represent multiple policies.

Source: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2

## Summary Statistics

In [24]:
# calculate summary statistics of each column, pandas df for readability
summarize_df = con.sql("""
    SELECT *
    FROM (SUMMARIZE claims)
""").df()

summarize_df.head()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,agricultureStructureIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2709121,0.00
1,asOfDate,TIMESTAMP WITH TIME ZONE,2019-09-19 02:12:43.388-04,2025-04-04 18:15:24.042001-04,534590,None,None,None,None,None,2709121,0.00
2,basementEnclosureCrawlspaceType,SMALLINT,0,4,4,1.1774572680931834,1.0643195971832151,0,1,2,2709121,69.82
3,policyCount,SMALLINT,1,1090,399,1.3080947657930377,6.775402579907944,1,1,1,2709121,0.00
4,crsClassificationCode,SMALLINT,1,10,10,6.631530645809065,1.5335857456943742,5,7,8,2709121,79.69


In [25]:
# slice through columns for readability
summarize_df.iloc[:15, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,agricultureStructureIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2709121,0.00
1,asOfDate,TIMESTAMP WITH TIME ZONE,2019-09-19 02:12:43.388-04,2025-04-04 18:15:24.042001-04,534590,None,None,None,None,None,2709121,0.00
2,basementEnclosureCrawlspaceType,SMALLINT,0,4,4,1.1774572680931834,1.0643195971832151,0,1,2,2709121,69.82
3,policyCount,SMALLINT,1,1090,399,1.3080947657930377,6.775402579907944,1,1,1,2709121,0.00
4,crsClassificationCode,SMALLINT,1,10,10,6.631530645809065,1.5335857456943742,5,7,8,2709121,79.69
5,dateOfLoss,DATE,1978-01-01,2025-04-04,17071,None,None,None,None,None,2709121,0.00
6,elevatedBuildingIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2709121,0.00
7,elevationCertificateIndicator,VARCHAR,1,E,9,None,None,None,None,None,2709121,77.92
8,elevationDifference,"DECIMAL(6,1)",-9989.0,998.0,374,1.2431787757753237,28.910725186550668,0,1,3,2709121,72.95
9,baseFloodElevation,"DECIMAL(6,1)",-9999.0,9998.0,10634,127.654352819797,771.7745761464929,7,9,14,2709121,75.67


In [26]:
# slice through columns for readability
summarize_df.iloc[15:30, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
15,numberOfFloorsInTheInsuredBuilding,SMALLINT,1,6,6,1.6940650617618291,0.8766043056392981,1,1,2,2709121,0.64
16,nonProfitIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2709121,0.00
17,obstructionType,SMALLINT,10,98,20,16.634793091457052,16.206249880346764,10,10,10,2709121,44.27
18,occupancyType,SMALLINT,1,19,14,2.0657381498359455,2.78172658519062,1,1,1,2709121,0.02
19,originalConstructionDate,DATE,1069-01-01,2025-01-01,32036,None,None,None,None,None,2709121,0.07
20,originalNBDate,DATE,0998-09-21,2025-05-03,17920,None,None,None,None,None,2709121,0.00
21,amountPaidOnBuildingClaim,"DECIMAL(12,2)",-201667.50,10741476.93,1339891,33314.29511758926,69675.86068590973,2366,9996,39330,2709121,20.96
22,amountPaidOnContentsClaim,"DECIMAL(12,2)",-80000.00,757048.95,491170,7231.943466524887,22283.450990218887,0,0,4905,2709121,20.96
23,amountPaidOnIncreasedCostOfComplianceClaim,"DECIMAL(12,2)",-6450.00,60000.00,8767,444.22112924451466,3431.6913221207583,0,0,0,2709121,20.96
24,postFIRMConstructionIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2709121,0.00


In [27]:
# slice through columns for readability
summarize_df.iloc[30:45, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
30,primaryResidenceIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2709121,0.00
31,buildingDamageAmount,BIGINT,0,927700000,211753,38122.347693035474,798675.4138763554,3542,11712,42974,2709121,22.13
32,buildingDeductibleCode,VARCHAR,0,H,15,None,None,None,None,None,2709121,11.63
33,netBuildingPaymentAmount,"DECIMAL(12,2)",-201667.50,10741476.93,1339823,26301.787513071584,63358.23914674135,6,4960,26762,2709121,0.00
34,buildingPropertyValue,BIGINT,0,9950000000,441933,6922338.358790468,196437513.99193782,61544,112386,195959,2709121,22.13
35,causeOfDamage,VARCHAR,0,Z,17,None,None,None,None,None,2709121,1.41
36,condominiumCoverageTypeCode,VARCHAR,A,U,5,None,None,None,None,None,2709121,1.38
37,contentsDamageAmount,BIGINT,0,19230507,104347,18437.408428089,84619.58349145742,1578,5756,18888,2709121,58.55
38,contentsDeductibleCode,VARCHAR,0,H,15,None,None,None,None,None,2709121,20.69
39,netContentsPaymentAmount,"DECIMAL(12,2)",-80000.00,757048.95,491141,5695.079842535642,20006.813626978263,0,0,2331,2709121,0.00


In [28]:
# slice through columns for readability
summarize_df.iloc[45:60, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
45,floodWaterDuration,SMALLINT,0,999,261,0.8054672691554323,14.715669857269845,0,0,0,2709121,10.17
46,floodproofedIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2709121,0.00
47,floodEvent,VARCHAR,2021 Mid-Spring Severe Storms,Yellowstone Flooding,180,None,None,None,None,None,2709121,28.22
48,iccCoverage,INTEGER,0,30000,4,28032.09650669587,4715.213351036216,30000,30000,30000,2709121,35.92
49,netIccPaymentAmount,"DECIMAL(8,2)",-6450.00,60000.00,8724,349.80937312508377,3050.500765592021,0,0,0,2709121,0.00
50,nfipRatedCommunityNumber,VARCHAR,000000,999999,16336,None,None,None,None,None,2709121,0.00
51,nfipCommunityNumberCurrent,VARCHAR,0000,815000,12095,None,None,None,None,None,2709121,71.68
52,nfipCommunityName,VARCHAR,ABBEVILLE COUNTY *,"ZUMBRO FALLS, CITY OF",9888,None,None,None,None,None,2709121,69.78
53,nonPaymentReasonContents,VARCHAR,01,99,23,None,None,None,None,None,2709121,68.57
54,nonPaymentReasonBuilding,VARCHAR,01,99,23,None,None,None,None,None,2709121,77.93


In [29]:
# slice through columns for readability
summarize_df.iloc[60:, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
60,waterDepth,SMALLINT,-999,999,474,4.361710069195211,16.581298603547097,0,1,2,2709121,8.72
61,floodZoneCurrent,VARCHAR,A,X,62,None,None,None,None,None,2709121,71.72
62,buildingDescriptionCode,SMALLINT,1,21,18,1.3382434848433562,1.4997932430426113,1,1,1,2709121,64.01
63,rentalPropertyIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2709121,0.00
64,state,VARCHAR,AK,WY,57,None,None,None,None,None,2709121,0.00
65,reportedCity,VARCHAR,Currently Unavailable,Currently Unavailable,1,None,None,None,None,None,2709121,0.00
66,reportedZipCode,VARCHAR,,99999,26361,None,None,None,None,None,2709121,0.00
67,countyCode,VARCHAR,01001,78030,2950,None,None,None,None,None,2709121,2.30
68,censusTract,VARCHAR,01001020100,78030961200,61803,None,None,None,None,None,2709121,5.08
69,censusBlockGroupFips,VARCHAR,010010201001,780309612002,122092,None,None,None,None,None,2709121,5.08


## Highlighted Features: Amount Paid on Claims Summary Statistics
We will analyze these attributes extensively throughout this notebook.

**amountPaidOnBuildingClaim**: Dollar amount paid on the building claim. In some instances, a negative amount may appear which occurs when a check issued to a policy holder is not cashed and has to be re-issued.

**amountPaidOnContentsClaim**: Dollar amount paid on the contents claim. In some instances, a negative amount may appear, which occurs when a check issued to a policy holder is not cashed and has to be re-issued.

**amountPaidOnIncreasedCostOfComplianceClaim**: ICC coverage is one of several flood insurance resources for policyholders who need additional help rebuilding after a flood. It provides up to $30,000 to help cover the cost of mitigation measures that will reduce the flood risk.

Source: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2

In [30]:
# examine summary statistics on paid total claims
con.sql("""
    SELECT
        column_name, column_type, count, null_percentage,
        min, max, approx_unique,
        ROUND(avg::DOUBLE, 2) AS avg,
        ROUND(std::DOUBLE, 2) AS std,
        ROUND(q25::DOUBLE, 2) AS q25,
        ROUND(q50::DOUBLE, 2) AS q50,
        ROUND(q75::DOUBLE, 2) AS q75
    FROM
        (SUMMARIZE claims)
    WHERE column_name IN (
        'amountPaidOnBuildingClaim',
        'amountPaidOnContentsClaim',
        'amountPaidOnIncreasedCostOfComplianceClaim'
    )
""").df()

,column_name,column_type,count,null_percentage,min,max,approx_unique,avg,std,q25,q50,q75
0,amountPaidOnBuildingClaim,"DECIMAL(12,2)",2709121,20.96,-201667.50,10741476.93,1339891,33314.30,69675.86,2367.0,9993.0,39341.0
1,amountPaidOnContentsClaim,"DECIMAL(12,2)",2709121,20.96,-80000.00,757048.95,491170,7231.94,22283.45,0.0,0.0,4916.0
2,amountPaidOnIncreasedCostOfComplianceClaim,"DECIMAL(12,2)",2709121,20.96,-6450.00,60000.00,8767,444.22,3431.69,0.0,0.0,0.0


In [31]:
# top 10 paid total claim records
con.sql("""
    SELECT
        dateOfLoss,
        floodEvent,
        state,
        policyCount,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim, 0)::INT AS paidTotalClaim,
        ROUND(amountPaidOnBuildingClaim, 0)::INT AS amountPaidOnBuildingClaim,
        ROUND(amountPaidOnContentsClaim, 0)::INT AS amountPaidOnContentsClaim,
        ROUND(amountPaidOnIncreasedCostOfComplianceClaim, 0)::INT AS amountPaidICC
    FROM
        claims
    ORDER BY
        paidTotalClaim DESC
    LIMIT 10
""").df()

,dateOfLoss,floodEvent,state,policyCount,paidTotalClaim,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidICC
0,2022-09-28,Hurricane Ian,FL,198,10841477,10741477,100000,0
1,2005-08-29,Hurricane Katrina,MS,48,10000000,10000000,0,0
2,2012-10-29,Hurricane Sandy,NY,184,9467720,9467720,0,0
3,2004-09-15,Hurricane Ivan,FL,64,9169507,9100033,39474,30000
4,2001-06-09,Tropical Storm Allison,TX,233,9023558,8973270,50288,0
5,2022-09-28,Hurricane Ian,FL,36,9000000,9000000,0,0
6,2022-09-28,Hurricane Ian,FL,72,8758773,8758773,0,0
7,2022-09-28,Hurricane Ian,FL,54,8734196,8734196,0,0
8,2005-08-29,Hurricane Katrina,MS,71,8386100,8349000,37100,0
9,2022-09-28,Hurricane Ian,FL,51,7778937,7778937,0,0


In [32]:
# top 10 paid total claim records for only one policy
con.sql("""
    SELECT
        dateOfLoss,
        floodEvent,
        state,
        policyCount,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim, 0)::INT AS paidTotalClaim,
        ROUND(amountPaidOnBuildingClaim, 0)::INT AS amountPaidOnBuildingClaim,
        ROUND(amountPaidOnContentsClaim, 0)::INT AS amountPaidOnContentsClaim,
        ROUND(amountPaidOnIncreasedCostOfComplianceClaim, 0)::INT AS amountPaidICC
    FROM
        claims
    WHERE
        policyCount == 1
    ORDER BY
        paidTotalClaim DESC
    LIMIT 10
""").df()

,dateOfLoss,floodEvent,state,policyCount,paidTotalClaim,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidICC
0,2008-06-13,None,IA,1,1250000,500000,750000,0
1,2023-04-13,April Florida Flooding,FL,1,1065210,1000000,65210,0
2,2016-05-24,None,AR,1,1054205,500000,554205,0
3,2012-10-29,Hurricane Sandy,NJ,1,1050000,500000,550000,0
4,2012-10-29,Hurricane Sandy,NJ,1,1039512,500000,539512,0
5,2005-08-29,Hurricane Katrina,LA,1,1000000,500000,500000,0
6,2005-08-29,Hurricane Katrina,LA,1,1000000,500000,500000,0
7,2005-08-29,Hurricane Katrina,LA,1,1000000,500000,500000,0
8,2005-08-29,Hurricane Katrina,LA,1,1000000,500000,500000,0
9,2005-08-29,Hurricane Katrina,LA,1,1000000,500000,500000,0


## Additional Attributes: Building Damage, Values, and Replacement Costs Summary Statistics

In [33]:
# examine summary statistics on building damage amounts, property values and replacement costs columns
con.sql("""
    SELECT
        column_name, column_type, count, null_percentage,
        min, max, approx_unique,
        ROUND(avg::DOUBLE, 2) AS avg,
        ROUND(std::DOUBLE, 2) AS std,
        ROUND(q25::DOUBLE, 2) AS q25,
        ROUND(q50::DOUBLE, 2) AS q50,
        ROUND(q75::DOUBLE, 2) AS q75,
    FROM
        (SUMMARIZE claims)
    WHERE column_name IN (
        'buildingPropertyValue',
        'buildingReplacementCost',
        'buildingDamageAmount'
    )
""").df()

,column_name,column_type,count,null_percentage,min,max,approx_unique,avg,std,q25,q50,q75
0,buildingPropertyValue,BIGINT,2709121,22.13,0,9950000000,441933,6922338.36,1.964375e+08,61528.0,112567.0,195963.0
1,buildingReplacementCost,BIGINT,2709121,22.13,0,9999000000,471726,8306497.31,2.309959e+08,0.0,124925.0,233038.0
2,buildingDamageAmount,BIGINT,2709121,22.13,0,927700000,211753,38122.35,7.986754e+05,3534.0,11698.0,42916.0


## Additional Attributes: Elevation and Water Depth Summary Statistics

In [34]:
# examine summary statisitics on elevation and water depth columns
con.sql("""
    SELECT
        column_name, column_type, count, null_percentage,
        min, max, approx_unique,
        ROUND(avg::DOUBLE, 2) AS avg,
        ROUND(std::DOUBLE, 2) AS std,
        ROUND(q25::DOUBLE, 2) AS q25,
        ROUND(q50::DOUBLE, 2) AS q50,
        ROUND(q75::DOUBLE, 2) AS q75
    FROM
        (SUMMARIZE claims)
    WHERE column_name IN (
        'baseFloodElevation',
        'waterDepth',
        'lowestAdjacentGrade',
        'lowestFloorElevation',
        'elevationDifference',
    )
""").df()

,column_name,column_type,count,null_percentage,min,max,approx_unique,avg,std,q25,q50,q75
0,elevationDifference,"DECIMAL(6,1)",2709121,72.95,-9989.0,998.0,374,1.24,28.91,0.0,1.0,3.0
1,baseFloodElevation,"DECIMAL(6,1)",2709121,75.67,-9999.0,9998.0,10634,127.65,771.77,7.0,9.0,14.0
2,lowestAdjacentGrade,"DECIMAL(6,1)",2709121,81.10,-99999.9,9998.9,12903,51.55,1433.71,3.0,6.0,11.0
3,lowestFloorElevation,"DECIMAL(6,1)",2709121,76.36,-9999.0,9998.9,13543,98.12,563.77,7.0,10.0,17.0
4,waterDepth,SMALLINT,2709121,8.72,-999,999,474,4.36,16.58,0.0,1.0,2.0


# Analysis
Note: All dollar amounts are reported in nominal terms and have not been adjusted for inflation.

## Claims Statistics

In [35]:
con.sql("""
    SELECT
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
""").df()

,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,2709121,87775002888,71337632479,15486136606,951233804


## Top 20 States by Number of Claims

In [36]:
con.sql("""
    SELECT
        state,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        state
    ORDER BY
        countClaims DESC
    LIMIT 20
""").df()

,state,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,LA,484331,20941810130,16522944159,4141133124,277732847
1,FL,447060,18117024733,15596041706,2481613265,39369762
2,TX,392340,17237432271,13268777147,3910460117,58195007
3,NJ,201261,6475715306,5352639021,872538667,250537618
4,NY,174857,5736218734,4961899549,720641697,53677488
5,NC,108760,2218805720,1879653466,302038024,37114230
6,PA,76681,1434787250,1126139631,293469016,15178602
7,MS,64161,3123646782,2386960573,675734337,60951873
8,CA,53261,743308260,637475392,104766571,1066297
9,IL,52516,590239993,493925645,81073490,15240858


## Top 20 States by Total Paid Claims

In [37]:
con.sql("""
    SELECT
        state,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        state
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,state,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,LA,484331,20941810130,16522944159,4141133124,277732847
1,FL,447060,18117024733,15596041706,2481613265,39369762
2,TX,392340,17237432271,13268777147,3910460117,58195007
3,NJ,201261,6475715306,5352639021,872538667,250537618
4,NY,174857,5736218734,4961899549,720641697,53677488
5,MS,64161,3123646782,2386960573,675734337,60951873
6,NC,108760,2218805720,1879653466,302038024,37114230
7,PA,76681,1434787250,1126139631,293469016,15178602
8,AL,44787,1192301060,967013188,212400512,12887360
9,SC,49500,1034748138,897285254,133613083,3849802


## Top 20 Counties by Number of Claims

In [38]:
con.sql("""
    SELECT
        state,
        countyCode,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        state, countyCode
    ORDER BY
        countClaims DESC
    LIMIT 20
""").df()

,state,countyCode,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,TX,48201,170708,8756942570,6801142758,1935750334,20049478
1,LA,22051,135450,3581401865,2667419595,852349104,61633166
2,LA,22071,127165,7299175307,5895159387,1293522405,110493515
3,FL,12086,61926,848437039,666464877,181972162,0
4,TX,48167,60585,2469311013,1898427427,549912817,20970769
5,NJ,34029,52799,2608089753,2160918465,275259388,171911899
6,FL,12103,51565,3251421135,2800444832,450424796,551508
7,NY,36059,51485,2280303382,1973061923,290486742,16754717
8,FL,12071,48350,3817648348,3422391501,392478724,2778124
9,LA,22103,38317,1778849652,1381031308,383543475,14274869


## Top 20 Counties by Total Paid Claims

In [39]:
con.sql("""
    SELECT
        state,
        countyCode,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        state, countyCode
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,state,countyCode,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,TX,48201,170708,8756942570,6801142758,1935750334,20049478
1,LA,22071,127165,7299175307,5895159387,1293522405,110493515
2,FL,12071,48350,3817648348,3422391501,392478724,2778124
3,LA,22051,135450,3581401865,2667419595,852349104,61633166
4,FL,12103,51565,3251421135,2800444832,450424796,551508
5,NJ,34029,52799,2608089753,2160918465,275259388,171911899
6,TX,48167,60585,2469311013,1898427427,549912817,20970769
7,NY,36059,51485,2280303382,1973061923,290486742,16754717
8,LA,22087,23915,2245129592,1680126546,553968821,11034225
9,LA,22103,38317,1778849652,1381031308,383543475,14274869


## Top 20 Flood Events by Number of Claims

In [40]:
con.sql("""
    SELECT
        floodEvent,
        yearOfLoss,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    WHERE
        floodEvent NOT NULL
    GROUP BY
        floodEvent, yearOfLoss
    ORDER BY
        countClaims DESC
    LIMIT 20
""").df()

,floodEvent,yearOfLoss,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,Hurricane Katrina,2005,208348,16261697056,12659081935,3360020221,242594900
1,Hurricane Sandy,2012,144848,8957334864,7707744797,951637071,297952996
2,Hurricane Harvey,2017,92397,9055711922,6925370027,2115077279,15264616
3,Hurricane Ike,2008,58126,2702511916,2073801567,577791589,50918760
4,Hurricane Helene,2024,57738,5802358735,5094231421,707558195,569120
5,Hurricane Irene,2011,52493,1347399996,1139897981,183189121,24312894
6,Hurricane Ian,2022,48755,4830393644,4299101444,528260936,3031263
7,Flooding,1995,47489,731729839,554632707,177097132,0
8,Tropical Storm Allison,2001,35561,1104979705,820333554,279280479,5365672
9,Hurricane Irma,2017,33339,1114746473,945893687,162153778,6699008


## Top 20 Flood Events by Total Paid Claims

In [41]:
con.sql("""
    SELECT
        floodEvent,
        yearOfLoss,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    WHERE
        floodEvent NOT NULL
    GROUP BY
        floodEvent, yearOfLoss
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,floodEvent,yearOfLoss,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,Hurricane Katrina,2005,208348,16261697056,12659081935,3360020221,242594900
1,Hurricane Harvey,2017,92397,9055711922,6925370027,2115077279,15264616
2,Hurricane Sandy,2012,144848,8957334864,7707744797,951637071,297952996
3,Hurricane Helene,2024,57738,5802358735,5094231421,707558195,569120
4,Hurricane Ian,2022,48755,4830393644,4299101444,528260936,3031263
5,Hurricane Ike,2008,58126,2702511916,2073801567,577791589,50918760
6,Mid-summer severe storms,2016,30018,2533534783,2175218939,347276988,11038856
7,Hurricane Irene,2011,52493,1347399996,1139897981,183189121,24312894
8,Hurricane Ida,2021,28317,1347046654,1115954806,229872957,1218891
9,Hurricane Ivan,2004,20137,1325419294,1083795424,221959720,19664150


## Top 20 States and Flood Events by Number of Claims

In [42]:
con.sql("""
    SELECT
        state,
        floodEvent,
        yearOfLoss,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    WHERE
        floodEvent NOT NULL
    GROUP BY ALL
    ORDER BY
        countClaims DESC
    LIMIT 20
""").df()

,state,floodEvent,yearOfLoss,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,LA,Hurricane Katrina,2005,176276,13347164644,10441743931,2721451049,183969663
1,TX,Hurricane Harvey,2017,91871,9040370286,6912549831,2112555840,15264616
2,NJ,Hurricane Sandy,2012,74983,4372246883,3673280769,457640033,241326081
3,NY,Hurricane Sandy,2012,57405,4214945920,3702376632,465410811,47158477
4,FL,Hurricane Helene,2024,54362,5568316047,4888593402,679153525,569120
5,FL,Hurricane Ian,2022,47491,4784448012,4257475620,524106129,2866263
6,TX,Hurricane Ike,2008,44095,2230307106,1723323896,481446923,25536286
7,LA,Flooding,1995,36820,594357775,442113752,152244023,0
8,LA,Mid-summer severe storms,2016,30018,2533534783,2175218939,347276988,11038856
9,FL,Hurricane Irma,2017,28756,985659492,834513863,145158635,5986994


## Top 20 States and Flood Events by Total Paid Claims

In [43]:
con.sql("""
    SELECT
        state,
        floodEvent,
        yearOfLoss,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    WHERE
        floodEvent NOT NULL
    GROUP BY ALL
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,state,floodEvent,yearOfLoss,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,LA,Hurricane Katrina,2005,176276,13347164644,10441743931,2721451049,183969663
1,TX,Hurricane Harvey,2017,91871,9040370286,6912549831,2112555840,15264616
2,FL,Hurricane Helene,2024,54362,5568316047,4888593402,679153525,569120
3,FL,Hurricane Ian,2022,47491,4784448012,4257475620,524106129,2866263
4,NJ,Hurricane Sandy,2012,74983,4372246883,3673280769,457640033,241326081
5,NY,Hurricane Sandy,2012,57405,4214945920,3702376632,465410811,47158477
6,LA,Mid-summer severe storms,2016,30018,2533534783,2175218939,347276988,11038856
7,MS,Hurricane Katrina,2005,19051,2521512578,1910623703,558996059,51892816
8,TX,Hurricane Ike,2008,44095,2230307106,1723323896,481446923,25536286
9,FL,Hurricane Milton,2024,21397,1000246733,903924271,96292462,30000


## Top 20 Rated Flood Zones by Number of Claims
From the Data Dictionary:
>Formerly called floodZone. NFIP Flood Zone derived from the Flood Insurance Rate Map (FIRM) used to rate the insured property. A - Special Flood with no Base Flood Elevation on FIRM; AE, A1-A30 - Special Flood with Base Flood Elevation on FIRM; A99 - Special Flood with Protection Zone; AH, AHB* - Special Flood with Shallow Ponding; AO, AOB* - Special Flood with Sheet Flow; X, B - Moderate Flood from primary water source. Pockets of areas subject to drainage problems; X, C - Minimal Flood from primary water source. Pockets of areas subject to drainage problems; D - Possible Flood; V - Velocity Flood with no Base Flood Elevation on FIRM; VE, V1-V30 - Velocity Flood with Base Flood Elevation on FIRM; AE, VE, X - New zone designations used on new maps starting January 1, 1986, in lieu of A1-A30, V1-V30, and B and C; AR - A Special Flood Hazard Area that results from the decertification of a previously accredited flood protection system that is determined to be in the process of being restored to provide base flood protection; AR Dual Zones - (AR/AE, AR/A1-A30, AR/AH, AR/AO, AR/A) Areas subject to flooding from failure of the flood protection system (Zone AR) which also overlap an existing Special Flood Hazard Area as a dual zone; *AHB, AOB, ARE, ARH, ARO, and ARA are not risk zones shown on a map, but are acceptable values for rating purposes*

In [44]:
con.sql("""
    SELECT
        ratedFloodZone as ratedFloodZone,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY ratedFloodZone
    ORDER BY
        countClaims DESC
    LIMIT 20
""").df()

,ratedFloodZone,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,AE,932221,41926630383,35259116973,6149060389,518453021
1,X,403445,13706775072,10612591078,3065449697,28734297
2,A,200785,3240569796,2585005476,618290707,37273612
3,C,163799,2853959043,2119933111,723910551,10115382
4,None,139027,696659870,491536385,205049316,74169
5,B,114963,3827228294,2836476360,971915047,18836887
6,A04,51803,2186529083,1801390834,364498523,20639726
7,A01,51360,1526374899,1189314596,321004209,16056094
8,A05,48583,1274240093,1058736120,186582282,28921691
9,VE,47851,1920438099,1709256146,193137177,18044776
